## 构建一个简单的CNN进行预测

模型：
1. 类型 尺寸/步长/数量 输入尺寸
2. 卷积 5x5/1/32 28x28x1
3. 池化 2x2/2/1  28x28x32
4. 卷积 5x5/1/64 14x14x32
5. 池化 2x2/2/1  14x14x64
6. 全连接 7x7x64 X 1024
7. relu
8. dropout
9. 全连接1024 X 10
10. softmax

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
# import data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

#test data
testX = mnist.test.images
testY = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#初始化权重
def weight_variable(shape):
    data = tf.truncated_normal(stddev=0.01,shape=shape)
    return tf.Variable(data)

#初始化偏置单元
def bais_variable(shape):
    data = tf.constant(0.01,shape=shape)
    return tf.Variable(data)

In [3]:
#这里的padding=SAME参考https://www.tensorflow.org/api_guides/python/nn#Notes_on_SAME_Convolution_Padding
#简易来说目的是为了使得卷积输出的size为原size除以步长的上取整
#函数使用参见https://www.tensorflow.org/api_docs/python/tf/nn/conv2d
#x为一个4维张量，[batch, in_height, in_width, in_channels]
#W为一个4维张量，[filter_height, filter_width, in_channels, out_channels]
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [4]:
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])
x_image = tf.reshape(x,[-1,28,28,1])

W1 = weight_variable([5,5,1,32])
b1 = bais_variable([32])
W2 = weight_variable([5,5,32,64])
b2 = bais_variable([64])
W3 = weight_variable([7*7*64,1024])
b3 = bais_variable([1024])
W4 = weight_variable([1024,10])
b4 = bais_variable([10])

keep_prob = tf.placeholder(tf.float32)

In [5]:
h_conv1 = tf.nn.relu(conv2d(x_image,W1)+b1)
h_pool1 = max_pool_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1,W2)+b2)
h_pool2 = max_pool_2x2(h_conv2)

x_pool2 = tf.reshape(h_pool2,[-1,64*7*7])
fc = tf.nn.relu(tf.matmul(x_pool2,W3)+b3)

fc_dropout = tf.nn.dropout(fc,keep_prob=keep_prob)

model = tf.nn.softmax(tf.matmul(fc_dropout,W4)+b4)

In [6]:
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model),reduction_indices=[1]))

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(model,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [8]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for i in range(10000):
    batch_x,batch_y = mnist.train.next_batch(50)
    sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:0.75})
    if i%100==0:
        c,a = sess.run([loss,accuracy],feed_dict={x:testX,y:testY,keep_prob:1.})
        print("step %d:loss=%.3f,accuracy=%.3f"%(i,c,a))

step 0:loss=2.302,accuracy=0.101
step 100:loss=0.913,accuracy=0.737
step 200:loss=0.451,accuracy=0.865
step 300:loss=0.350,accuracy=0.901
step 400:loss=0.295,accuracy=0.915
step 500:loss=0.263,accuracy=0.919
step 600:loss=0.238,accuracy=0.926
step 700:loss=0.216,accuracy=0.934
step 800:loss=0.192,accuracy=0.941
step 900:loss=0.172,accuracy=0.947
step 1000:loss=0.170,accuracy=0.947
step 1100:loss=0.158,accuracy=0.952
step 1200:loss=0.130,accuracy=0.961
step 1300:loss=0.129,accuracy=0.961
step 1400:loss=0.117,accuracy=0.963
step 1500:loss=0.120,accuracy=0.964
step 1600:loss=0.108,accuracy=0.966
step 1700:loss=0.102,accuracy=0.968
step 1800:loss=0.093,accuracy=0.972
step 1900:loss=0.092,accuracy=0.971
step 2000:loss=0.085,accuracy=0.974
step 2100:loss=0.090,accuracy=0.973
step 2200:loss=0.083,accuracy=0.975
step 2300:loss=0.075,accuracy=0.978
step 2400:loss=0.072,accuracy=0.977
step 2500:loss=0.072,accuracy=0.978
step 2600:loss=0.078,accuracy=0.976
step 2700:loss=0.068,accuracy=0.978
step